In [2]:
import os
import sys
src_path = os.path.abspath('../src')
sys.path.append(src_path)
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd
import numpy as np

from src import data
from src import portfolio
from src import efficient_frontier
from src import plot
from src import capm
import yfinance as yf
from scipy.optimize import Bounds, LinearConstraint
import statsmodels.api as smf

In [3]:
weights = np.array([1/3,1/3,1/3])
min_esg_score = 1100
max_esg_score = 2000
df = pd.read_excel(r"../data/ESG_DATA_S&P500.xlsx")

dates = ['2021-01-01','2022-01-01']
start_year = dates[0]
end_year = dates[1]

Bounds = Bounds(0,1) #How willing we are to go short and to invest in one particular stock
Wanted_return = 0.20 #Only used when using the wanted_return constraint
maximum_risk = 0.10 #Either used when using maximum risk constraint or cmle portfolio
sharpe_type = "No_extra_constraint"  # rename to constraint, can be either Wanted_return or Maximum_risk or No_extra_constraint
risk_free_rate = 0.02

In [4]:
esg_data = data.esg_score_weight(df,weights,min_esg_score) #ESG_DATA filtering function
full_data = data.stock_monthly_close(esg_data,dates) #Gives us the data we need from the stocks with relevant ESG_scores
prices,esgdata = data.seperate_full_data(full_data) #Gives us price data and esg data to use in future functions
prices = prices[['MMM','AOS','ABT','ACN','ADM','ADP']]
esgdata = esgdata[['MMM_weighted','AOS_weighted','ABT_weighted','ACN_weighted','ADM_weighted','ADP_weighted']]

pct_returns = data.pct_returns_from_prices(prices) #Gives us the returns in pct for our stocks
parameters = portfolio.efficient_frontier_solo(pct_returns,Bounds, sharpe_type,dates[0],dates[1], Wanted_return, maximum_risk) #Different amount of rows from ESG_DATA as it drops all na. Gives us the efficient frontier for the given tie frame
weights_of_each_portfolio = efficient_frontier.weights_of_portfolio(prices,parameters)
sp500 = data.data_for_beta(prices,['SPY'],dates)
pct_returns_sp500 = data.pct_returns_from_prices(sp500)
market_name = 'SPY'


Min. Risk = 14.273% => Return: 27.602%  Sharpe Ratio = 1.93
Max. Sharpe Ratio = 3.16 => Return: 52.39%  Risk: 16.604%


In [13]:
print(sp500)
betas_of_portfolio = capm.calculate_portfolio_beta(pct_returns_sp500,prices,weights_of_each_portfolio,market_name)
pct_returns_sp500
prices
#1.3142


                                  MMM        AOS         ABT         ACN  \
Date                                                                       
2021-01-01 00:00:00-05:00  175.660004  54.299999  123.589996  241.919998   
2021-02-01 00:00:00-05:00  175.059998  59.369999  119.779999  250.899994   
2021-03-01 00:00:00-05:00  192.679993  67.610001  119.839996  276.250000   
2021-04-01 00:00:00-04:00  197.139999  67.750000  120.080002  289.970001   
2021-05-01 00:00:00-04:00  203.039993  71.070000  116.650002  282.160004   
2021-06-01 00:00:00-04:00  198.630005  72.059998  115.930000  294.790009   
2021-07-01 00:00:00-04:00  197.940002  70.330002  120.980003  317.679993   
2021-08-01 00:00:00-04:00  194.740005  72.720001  126.370003  336.559998   
2021-09-01 00:00:00-04:00  175.419998  61.070000  118.129997  319.920013   
2021-10-01 00:00:00-04:00  178.679993  73.070000  128.889999  358.790009   
2021-11-01 00:00:00-04:00  170.039993  79.050003  125.769997  357.399994   
2021-12-01 0

,MMM,AOS,ABT,ACN,ADM,ADP
Date,,,,,,
2021-01-01 00:00:00-05:00,175.660004,54.299999,123.589996,241.919998,50.009998,165.119995
2021-02-01 00:00:00-05:00,175.059998,59.369999,119.779999,250.899994,56.580002,174.020004
2021-03-01 00:00:00-05:00,192.679993,67.610001,119.839996,276.250000,57.000000,188.470001
2021-04-01 00:00:00-04:00,197.139999,67.750000,120.080002,289.970001,63.130001,186.990005
2021-05-01 00:00:00-04:00,203.039993,71.070000,116.650002,282.160004,66.529999,196.020004
2021-06-01 00:00:00-04:00,198.630005,72.059998,115.930000,294.790009,60.599998,198.619995
2021-07-01 00:00:00-04:00,197.940002,70.330002,120.980003,317.679993,59.720001,209.630005
2021-08-01 00:00:00-04:00,194.740005,72.720001,126.370003,336.559998,60.000000,209.039993
2021-09-01 00:00:00-04:00,175.419998,61.070000,118.129997,319.920013,60.009998,199.919998


In [10]:
weights_of_each_portfolio.to_numpy()


array([[8.60460441e-17, 5.51118763e-17, 0.00000000e+00, 4.40978139e-01,
        6.14734475e-02, 4.97548413e-01]])

In [15]:
capm.calculate_portfolio_beta_ols(pct_returns_sp500,prices,weights_of_each_portfolio,'SPY')

const   -0.025784
SPY      1.196580
dtype: float64
const   -0.003495
SPY      2.134078
dtype: float64
const   -0.015482
SPY      1.226955
dtype: float64
const    0.014506
SPY      1.596504
dtype: float64
const    0.008852
SPY      0.890518
dtype: float64
const    0.015584
SPY      0.961083
dtype: float64
[1.19658008100946, 2.134077761938658, 1.2269547650734427, 1.5965039255509357, 0.8905179338255013, 0.9610826308848972]


1.2369516759663992